# TITLE

#### Group name: 
- Group_1

#### Group Members:
- Omer Yalavac
- Vincent Tang
- Rhyme Bulbul

# Table of Contents
1. [Introduction](#Introduction)
2. [Overview](#Overview)
3. [Third Example](#third-example)
4. [Fourth Example](#fourth-examplehttpwwwfourthexamplecom)

# Introduction <a name="Introduction"></a>

The objective of this project is to 

# Overview <a name="Overview"></a>

### Data Source

Dataset was collected from http://archive.ics.uci.edu/ml/datasets/Student+Performance, using 'student-por.csv'.
Our dataset has 649 instances, with 33 attributes, which can be verified with <code>df.dtypes</code>.

In [1]:
import pandas as pd

df = pd.read_csv('student-por.csv', sep=';')

Our dataset was only one column of data including the column title with only the semi-colon (;) symbol seperating values, and so it needed to be seperated when being read in order to create the seperate columns with their respective row values using 
```
sep = ';'
```
---
This is how our dataset currently looks:

In [2]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


### Project Objective

### Target Feature

### Descriptive Features

The variable descriptions below are from the <code>student-por</code> file:

- <code>school</code>- student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
- <code>sex</code> - student's sex (binary: 'F' - female or 'M' - male)
- <code>age</code> - student's age (numeric: from 15 to 22)
- <code>address</code> - student's home address type (binary: 'U' - urban or 'R' - rural)
- <code>famsize</code> - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
- <code>Pstatus</code> - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
- <code>Medu</code> - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
- <code>Fedu</code> - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
- <code>Mjob</code> - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- <code>Fjob</code> - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- <code>reason</code> - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
- <code>guardian</code> - student's guardian (nominal: 'mother', 'father' or 'other')
- <code>traveltime</code> - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
- <code>studytime</code> - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
- <code>failures</code> - number of past class failures (numeric: n if 1<=n<3, else 4)
- <code>schoolsup</code> - extra educational support (binary: yes or no)
- <code>famsup</code> - family educational support (binary: yes or no)
- <code>paid</code> - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
- <code>activities</code> - extra-curricular activities (binary: yes or no)
- <code>nursery</code> - attended nursery school (binary: yes or no)
- <code>higher</code> - wants to take higher education (binary: yes or no)
- <code>internet</code> - Internet access at home (binary: yes or no)
- <code>romantic</code> - with a romantic relationship (binary: yes or no)
- <code>famrel</code> - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
- <code>freetime</code> - free time after school (numeric: from 1 - very low to 5 - very high)
- <code>goout</code> - going out with friends (numeric: from 1 - very low to 5 - very high)
- <code>Dalc</code> - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
- <code>Walc</code> - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
- <code>health</code> - current health status (numeric: from 1 - very bad to 5 - very good)
- <code>absences</code> - number of school absences (numeric: from 0 to 93) 

### these grades are related with the course subject, Math or Portuguese:
- <code>G1</code> - first period grade (numeric: from 0 to 20)
- <code>G2</code> - second period grade (numeric: from 0 to 20)
- <code>G3</code> - final grade (numeric: from 0 to 20, output target)

# Data Preparation

## Preliminaries

In [3]:
# Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import warnings
###
warnings.filterwarnings('ignore')
###
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
plt.style.use("ggplot")

## Data Cleaning and Transformation

In [4]:
print(f"Shape of the dataset is {df.shape} \n")
print(f"Data types are below where 'object' indicates a string type: ")
print(df.dtypes)



Shape of the dataset is (649, 33) 

Data types are below where 'object' indicates a string type: 
school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
G1             int64
G2             int64
G3             int64
dtype: object
